In [ ]:
import pandas as pd
import os
import re
import json
os.environ["OPENAI_API_KEY"] = "#####"
from openai import OpenAI
client = OpenAI()

def get_json(content):
    #get json part
    json_text = re.search(r'\{.*\}', content, re.DOTALL)
    # check json data
    if json_text:
        myjson = json_text.group()
        #print("Extracted JSON:", myjson)  # check extracted json
        try:
            data = json.loads(myjson)  # JSON parsing
            return data
        except json.JSONDecodeError as e:
            print("[jp] JSON parsing error:", e)
    else:
        print("[jp] JSON data not found")
    
    
    

def analyze_emotions(system_content,user_content):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ]
    )

    answer = response.choices[0].message.content #get json response
    return get_json(answer)

system_content = """
You will change the diary entry based to have very negative emotions shown. It can be a little bit mixed with neutral emotions.
If it has already negative emotions, add a little bit more. If there is very positive emotion, remove it. You can change the content but try to write is similar to the original.

Please respond strictly in JSON format with the following structure:

{
  "diary_entry": "...",
  "negative_diary_entry": ...
}
"""

user_content="""
diary_entry="I am so happy today")
"""

json_result = analyze_emotions(system_content,user_content)
print(json_result)

{'diary_entry': 'I am so happy today', 'negative_diary_entry': "I am really upset today. Everything just seems to be going wrong, and I can't shake off this feeling of disappointment that lingers over everything I've done."}


In [3]:
import pandas as pd

file = "test-created-164-answer-created-int.csv"
data = pd.read_csv(file)

print(data.head())


#for loop 'diary_entry' columns

# for each diary_entry get result from
json_result = analyze_emotions(system_content,user_content)
print(json_result)

# save it in csv file that has diary_entry, negative_diary_entry column


   Unnamed: 0                                        diary_entry  joy  \
0           0  As soon as I heard that Jimmy O. Yang was doin...    5   
1           1  It’s already been two years since I started le...    4   
2           2  I can’t understand why Sarah acted that way. I...    1   
3           3  I heard a few weeks ago that Squid Game 2 was ...    1   
4           4  Today started well with a quiet morning and a ...    3   

   sadness  anger  fear  disgust  surprise  
0        1      1     3        1         3  
1        2      1     2        1         3  
2        3      4     2        3         2  
3        2      3     1        3         2  
4        3      2     2        2         2  


In [ ]:
import pandas as pd
import os
import re
import json
os.environ["OPENAI_API_KEY"] = "#####"
from openai import OpenAI
client = OpenAI()

def get_json(content):
    # Extract JSON part
    json_text = re.search(r'\{.*\}', content, re.DOTALL)
    if json_text:
        myjson = json_text.group()
        try:
            data = json.loads(myjson)  # Parse JSON
            return data
        except json.JSONDecodeError as e:
            print("[jp] JSON parsing error:", e)
    else:
        print("[jp] JSON data not found")
    return None

def analyze_emotions(system_content, user_content):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ]
    )
    answer = response.choices[0].message.content  # Get JSON response
    return get_json(answer)

# Define system content
system_content = """
You will change the diary entry based to have very negative emotions shown. It can be a little bit mixed with neutral emotions.
If it has already negative emotions, add a little bit more. If there is very positive emotion, remove it. You can change the content but try to write is similar to the original.

Please respond strictly in JSON format with the following structure:

{
  "diary_entry": "...",
  "negative_diary_entry": "..."
}
"""

# Load data from CSV
file = "test-created-164-answer-created-int.csv"
data = pd.read_csv(file)

# Add a new column for negative diary entries
negative_entries = []

for i, row in data.iterrows():
    print(i)
    diary_entry = row['diary_entry']
    user_content = f'diary_entry="{diary_entry}"'
    try:
        json_result = analyze_emotions(system_content, user_content)
        if json_result and "negative_diary_entry" in json_result:
            print(json_result["negative_diary_entry"])
            negative_entries.append(json_result["negative_diary_entry"])
        else:
            print("error")
            negative_entries.append("Error processing")
    except Exception as e:
        print(f"[jp] Error for row {i}: {e}")
        negative_entries.append("Error processing")

# Save results to a new CSV
data['negative_diary_entry'] = negative_entries
output_file = "diary_negative_entries.csv"
data.to_csv(output_file, index=False)
print(f"Processed data saved to {output_file}")

0
When I heard that Jimmy O. Yang was doing a nationwide tour in March, I reluctantly booked my ticket despite the fact it's still months away. I started watching his shorts every day, hoping to feel some excitement, but instead, I just feel indifferent and somewhat weary. The thought of fighting for parking nearby makes it even worse. I dread the idea of circling the lot tirelessly, only to end up irritated and annoyed. Parking is always such a nightmare. But then again, thinking positive rarely changes anything.
1
It's been a depressing two years since I started learning pottery from my teacher. I’ve long struggled with learning techniques for mixing different colors, and last week, I begrudgingly learned the last one. My teacher said there was nothing more to teach me, which left me feeling quite alone and unprepared. Reluctantly, I created an Instagram page to share my painfully mediocre pottery work. I doubt if anyone will be interested, and I’m uncertain if there will be any dema